In [1]:
from math import sin, cos, asin, atan2, tan, pi, acos
from dataclasses import dataclass
from datetime import datetime, timedelta
from typing import Optional
import matplotlib.pyplot as plt
import numpy as np
import importlib
import SunMoonCalculator

importlib.reload(SunMoonCalculator)

from SunMoonCalculator import SunMoonCalculator

In [2]:
@dataclass
class EclipticCoordinates:
    #  longitute [rad] 
    λ: float
    #  latitude [rad] */
    β: float

@dataclass
class EquatorialCoordinates:
    # right ascension [rad] 
    α: float
    # declination [rad] 
    δ: float

@dataclass
class LongitudeAndMeanAnomaly:
    λ: float
    M: float

@dataclass
class MoonProperties:
    coords: EquatorialCoordinates
    phaseAngle: float
    MPrimM: float
    Ec: float

In [3]:
def dateDiff(datePast: datetime, dateFuture: datetime) -> float:
    return (dateFuture - datePast).total_seconds() / timedelta(days=1).total_seconds()

def degToRad(degrees: float):
	return degrees * (pi / 180)

def radToDeg(radians: float):
	return radians * (180 / pi)

def mapNr(x: float, fromMin: float, fromMax: float, toMin: float, toMax: float) -> float:
    return ((x - fromMin)/(fromMax - fromMin))*(toMax - toMin) + toMin

In [4]:
inline = False

if not inline:
    %matplotlib qt
else:
    %matplotlib inline

In [5]:
def eclipticToEquatorial(coords: EclipticCoordinates, date: datetime) -> EquatorialCoordinates:
    λ = coords.λ
    β = coords.β
    epoch = datetime(year=2000, month=1, day=1, hour=12, minute=0, second=0)

    T = dateDiff(epoch, date) / 36525

    DE = T * (46.815 + T * (0.0006 - T * 0.00181))

    ε = degToRad(23.439292 - (DE / 3600))

    α = atan2(sin(λ) * cos(ε) - tan(β) * sin(ε), cos(λ))

    if α < 0:
        α += 2 * pi

    return EquatorialCoordinates(
        α=α,
        δ=asin(sin(β) * cos(ε) + cos(β) * sin(ε) * sin(λ)),
    )

def sunLongitudeAndMeanAnomaly(date: datetime) -> LongitudeAndMeanAnomaly:
    epoch = datetime(year=2009, month=12, day=31, hour=0, minute=0, second=0)
    εg = 279.557208; # [deg]
    ϖg = 283.112438; # [deg]
    e = 0.016705;

    D = dateDiff(epoch, date)
    N = (360 / 365.242191) * D
    M = (N + εg - ϖg) % 360

    if M < 0:
        M += 360

    Ec = (360 / pi) * e * sin(degToRad(M));

    λ = (N + Ec + εg) % 360;

    if λ < 0:
        λ += 360;

    return LongitudeAndMeanAnomaly(λ, M)

def sunEquitorialCoordinates(date: datetime, l: Optional[LongitudeAndMeanAnomaly]):
    if l == None:
        l = sunLongitudeAndMeanAnomaly(date)

    return eclipticToEquatorial(EclipticCoordinates(λ=degToRad(l.λ), β=0), date)

def moonRelativeSize(Ec: float, MPrimM: float):
    e = 0.0549;
    return (1 - e ** 2) / (1 + e * cos(MPrimM + Ec))

def moonProperies(date: datetime, l: Optional[LongitudeAndMeanAnomaly]) -> MoonProperties:
    epoch = datetime(year=2009, month=12, day=31, hour=0, minute=0, second=0)

    if l == None:
        l = sunLongitudeAndMeanAnomaly(date)

    λSun = l.λ
    M = l.M

    D = dateDiff(epoch, date)

    l0 = 91.929336; # degrees
    P0 = 130.143076; # degrees
    N0 = 291.682547; # degrees
    i = 5.145396; # degrees

    Lm = (13.1763966 * D + l0) % 360
    Mm = (Lm - 0.1114041 * D - P0) % 360
    N = (N0 - 0.0529539 * D) % 360

    Ev = 1.2739 * sin(degToRad(2 * (Lm - λSun) - Mm))
    Ae = 0.1858 * sin(M)
    A3 = 0.37 * sin(M)
    MPrimM = Mm + Ev - Ae - A3

    Ec = 6.2886 * sin(degToRad(MPrimM))
    A4 = 0.214 * sin(2 * degToRad(MPrimM))
    LPrim = Lm + Ev + Ec - Ae + A4

    V = 0.6583 * sin(2 * degToRad(LPrim - λSun))
    LPrimPrim = LPrim + V
    Nd = N - 0.16 * sin(M)
    u = degToRad(LPrimPrim - Nd)

    y = sin(u) * cos(degToRad(i))
    x = cos(u)

    λ = (radToDeg(atan2(y, x)) + Nd) % 360
    β = asin(sin(u) * sin(degToRad(i)))

    return MoonProperties(
        coords=eclipticToEquatorial(EclipticCoordinates(λ=degToRad(λ), β=β), date),
        phaseAngle=degToRad(λ - λSun),
        MPrimM=MPrimM,
        Ec=Ec,
    )

def moonPositionAngle(sun: EquatorialCoordinates, moon: EquatorialCoordinates) -> float: 
    return atan2(
        sin(sun.δ) * sin(sun.α - moon.α),
        cos(moon.δ) * sin(sun.δ) - sin(moon.δ) * cos(sun.δ) * cos(sun.α - moon.α),
    )

def moonAnglesAndRelativeSize(date: datetime):
    λAndM = sunLongitudeAndMeanAnomaly(date)

    sunPos = sunEquitorialCoordinates(date, λAndM)
    props = moonProperies(date, λAndM)

    positionAngle = moonPositionAngle(sunPos, props.coords)
    relativeSize = moonRelativeSize(props.Ec, props.MPrimM)

    return (
        props.phaseAngle,
        positionAngle,
        relativeSize,
    )

start = datetime(year=2021, month=12, day=31)
end = datetime(year=2022, month=12, day=31)

x = np.arange(0, dateDiff(start, end), 0.1)
y = list(map(lambda d: moonAnglesAndRelativeSize(start + timedelta(days=d)), x))

def calcReference(d: float):
    calc = SunMoonCalculator(date=start + timedelta(days=d), obsLat=54, obsLon=19, obsAlt=1000)
    calc.calcSunAndMoon()

    return (calc.moonAge, calc.moon, calc.sun)

yRef = list(map(calcReference, x))


In [6]:
yRefPhase = [mapNr(age, 0, 30, 0, 2*pi) for (age, _, _) in yRef]
yRefAngularSize = [moon.angularRadius for (_, moon, _) in yRef]
yPos = [moonPositionAngle(EquatorialCoordinates(moon.rightAscension, moon.declination), EquatorialCoordinates(sun.rightAscension, sun.declination)) for (_, moon, sun) in yRef]

plt.figure('Phase angle')
plt.title('Phase angle')
plt.plot(x, yRefPhase)

plt.figure('Position angle')
plt.title('Position angle')
plt.plot(x, yPos)

plt.figure('Relative size')
plt.title('Relative size')
plt.plot(x, yRefAngularSize)

plt.show()
np.mean(yRefAngularSize)

0.004524851154581492